In [1]:
from omegaconf import OmegaConf

config_paths = [
        './config/dataset/waymo.yaml',
        './config/experiment/train.yaml',
        './config/model/yolox-s.yaml',
    ]

configs = [OmegaConf.load(path) for path in config_paths]
merged_conf = OmegaConf.merge(*configs)


In [2]:
from modules.fetch import fetch_data_module, fetch_model_module

data = fetch_data_module(merged_conf)
data.setup('fit')
model = fetch_model_module(merged_conf)
model.setup('fit')

train dataset size: 764615
valid dataset size: 199935
darknet
inchannels: (128, 256, 512)
PAFPN
strides: (8, 16, 32)
YOLOX-Head


In [3]:
data_iter = iter(data.train_dataloader())

In [4]:
import torch
import time

# データ取得速度測定
start_time = time.time()
for _ in range(100):  # サンプルを複数回取得
    sample = next(data_iter)
    image = sample['image'].to("cuda").to(torch.float32)
    label = sample['labels'].to("cuda").to(torch.float32)
end_time = time.time()

print(f"データ取得速度: {(end_time - start_time) / 100:.5f} 秒/サンプル")


Pre-transform time for index 41: 0.0521 seconds
Pre-transform time for index 84: 0.0649 seconds
Pre-transform time for index 51: 0.0709 seconds
Pre-transform time for index 94: 0.0655 secondsPre-transform time for index 45: 0.0862 seconds
Pre-transform time for index 38: 0.0870 seconds

Pre-transform time for index 36: 0.0903 seconds
Pre-transform time for index 138: 0.0943 seconds
Pre-transform time for index 59: 0.1021 secondsPre-transform time for index 69: 0.0991 seconds

Transform time for index 84: 0.0449 secondsTransform time for index 41: 0.0577 seconds
Pre-transform time for index 165: 0.1010 seconds
Total time for index 41: 0.1098 secondsTotal time for index 84: 0.1099 secondsPre-transform time for index 100: 0.1141 seconds



Transform time for index 51: 0.0513 seconds
Total time for index 51: 0.1222 secondsTransform time for index 38: 0.0416 secondsTransform time for index 45: 0.0416 seconds


Total time for index 45: 0.1278 seconds
Total time for index 38: 0.1286 seconds
T

データ取得速度: 0.17372 秒/サンプル


In [ ]:
model.to("cuda")
# 推論速度測定
start_time = time.time()
for _ in range(100):  # サンプルを複数回推論
    outputs = model(image, label)
end_time = time.time()

print(f"推論速度: {(end_time - start_time) / 100:.5f} 秒/サンプル")


In [ ]:
import time
import cv2

def measure_io_performance(image_path):
    # ファイル読み込み時間を計測
    start_time = time.time()
    with open(image_path, 'rb') as f:
        file_data = f.read()
    read_time = time.time() - start_time

    # 画像デコード時間を計測
    start_time = time.time()
    image = cv2.imdecode(np.frombuffer(file_data, np.uint8), cv2.IMREAD_COLOR)
    decode_time = time.time() - start_time

    # トータル時間を出力
    print(f"Read time: {read_time:.4f} seconds, Decode time: {decode_time:.4f} seconds")

# サンプル画像でテスト
measure_io_performance("/media/metis/AT_2TB/processed_waymo/val/10203656353524179475/images/FRONT/0000.jpg")
